This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-model-deploy-and-serve/xgboost-upload-tutorials).

## XGBoost Booster Multiclass Classification Softmax Example

The following tutorial demonstrates deploying and serving an XGBoost Booster Multiclass Classification Softmax model to Wallaroo.

The following XGBoost model types are supported by Wallaroo.  XGBoost models **not** supported by Wallaroo are supported via the [Arbitrary Python models, also known as Bring Your Own Predict (BYOP)](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/).

| XGBoost Model Type | Wallaroo Auto Packaging Supported |
|---|---|
| XGBClassifier | √ |
| XGBRegressor | √ |
| Booster Classifier | √ |
| Booster Classifier | √ |
| Booster Regressor | √ |
| Booster Random Forest Regressor | √ |
| Booster Random Forest Classifier | √ |
| XGBRFClassifier | √ |
| XGBRFRegressor | √ |
| XGBRanker\* | X |

* XGBRanker XGBoost models are currently supported via converting them to [BYOP](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/) models.

## Goal

Upload, deploy, and serve a sample XGBoost Booster Multiclass Classification Softmax model.

### Resources

This tutorial provides the following:

* Models:
  * `./models/booster_multi_classification_softmax.pkl`: The sample XGBoost model that receives the `sklearn.datasets.load_iris` dataset.

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
* A X64 Docker deployment to deploy the model on an edge location.
* The notebook "Wallaroo Run Anywhere Model Drift Observability with Assays: Preparation" has been run, and the model edge deployments executed.

## Steps

### Import Libraries

The first step is to import the libraries we will need.  See `./requirements.txt` for a list of additional libraries used with this tutorial.

In [9]:
import wallaroo
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.pipeline import Pipeline
import pyarrow as pa
from wallaroo.framework import Framework

import pickle
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from xgboost import train, DMatrix

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [10]:
wl = wallaroo.Client()

### Set Variables and Helper Functions

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

We'll set up some helper functions that will either use existing workspaces and pipelines, or create them if they do not already exist.

In [11]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

workspace_name = f'xgboost-booster-multiclass-classification-softmax'
pipeline_name = f'xgboost-booster-multiclass-classification-softmax'

model_name = 'booster-multiclass-classification-softmax'
model_file_name = './models/booster_multi_classification_softmax.pkl'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [12]:
workspace = get_workspace(workspace_name, wl)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Upload XGBoost Model

XGBoost models are uploaded to Wallaroo through the `wallaroo.client.Client.upload_model` method.

#### Upload XGBoost Model Parameters

The following parameters are available for XGBoost models.  

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Required*) | Set as `Framework.XGBOOST`. |
|`input_schema` | `pyarrow.lib.Schema` (*Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

#### Upload XGBoost Model Return

The following is returned with a successful model upload and conversion.

| Field | Type | Description |
|---|---|---|
| `name` | string | The name of the model. |
| `version` | string | The model version as a unique UUID. |
| `file_name` | string | The file name of the model as stored in Wallaroo. |
| `image_path` | string | The image used to deploy the model in the Wallaroo engine. |
| `last_update_time` | DateTime | When the model was last updated. |#

### Configure Input and Output Schemas

First we configure the input and output schemas in PyArrow format.

In [13]:
input_schema = pa.schema([
    pa.field('inputs', pa.list_(pa.float32(), list_size=4))
])

output_schema = pa.schema([
    pa.field('predictions', pa.float32()),
])

### Upload Model Example

With the input and output schemas defined, we now upload the XGBoost model.

In [14]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.XGBOOST, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.
Model is attempting loading to a native runtime.successful

Ready


Name,booster-multiclass-classification-softmax
Version,4087ee65-d601-4635-9fdb-b74b4fc531cc
File Name,booster_multi_classification_softmax.pkl
SHA,6ace49bb2a3514724937de8ec023a14c94ac71dde7dce220ad27e44d296b4b25
Status,ready
Image Path,None
Architecture,None
Acceleration,None
Updated At,2024-27-Mar 23:12:32


### Deploy Pipeline

With the model uploaded and packaged, we add the model as a pipeline model step.  For our deployment, we will set a minimum deployment configuration - this is the amount of resources the deployed pipeline uses from the cluster.

Once set, we deploy the pipeline, which allocates the assigned resources for the cluster and makes it available for inference requests.

In [15]:
pipeline.add_model_step(model)

deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [17]:
pipeline.undeploy()
pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.0.249',
   'name': 'engine-5f7dd49df8-g9frp',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'xgboost-booster-multiclass-classification-softmax',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'config': {'batch_config': None,
       'filter_threshold': None,
       'id': 40,
       'input_schema': '/////7AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAEAAAAEAAAAyP///wAAARAUAAAAJAAAAAQAAAABAAAAMAAAAAYAAABpbnB1dHMAAAAABgAIAAQABgAAAAQAAAAQABQACAAGAAcADAAAABAAEAAAAAAAAQMQAAAAHAAAAAQAAAAAAAAABAAAAGl0ZW0AAAYACAAGAAYAAAAAAAEAAAAAAA==',
       'model_version_id': 20,
       'output_schema': '/////4AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAEAAAAUAAAAEAAUAAgABgAHAAwAAAAQABAAAAAAAAEDEAAAACQAAAAEAAAAAAAAAAsAAABwcmVkaWN0aW9ucwAAAAYACAAGAAYAAAAAAAEAAAAAAA==',
       'runtime': 'onnx',
       'sidekick_uri': None,
       't

### Run Sample Inference

The dataset is from the `sklearn.datasets.load_iris` examples.  These are converted to a pandas DataFrame, that is submitted to the deployed model in Wallaroo for an inference request.

In [18]:
dataset = load_iris()
X, y = dataset.data, dataset.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
dtrain = DMatrix(X_train, label=y_train)
dtest = DMatrix(X_test, label=y_test)

data = dtest.get_data().todense()[:100]

In [19]:
import pandas as pd

dataframe = pd.DataFrame({"inputs": data.tolist()})

results = pipeline.infer(dataframe)
results

,time,in.inputs,out.predictions,anomaly.count
0,2024-03-27 23:14:49.948,"[6.0999999046, 2.7999999523, 4.6999998093, 1.2...",1.0,0
1,2024-03-27 23:14:49.948,"[5.6999998093, 3.7999999523, 1.7000000477, 0.3...",0.0,0
2,2024-03-27 23:14:49.948,"[7.6999998093, 2.5999999046, 6.9000000954, 2.2...",2.0,0
3,2024-03-27 23:14:49.948,"[6.0, 2.9000000954, 4.5, 1.5]",1.0,0
4,2024-03-27 23:14:49.948,"[6.8000001907, 2.7999999523, 4.8000001907, 1.3...",1.0,0
5,2024-03-27 23:14:49.948,"[5.4000000954, 3.4000000954, 1.5, 0.400000006]",0.0,0
6,2024-03-27 23:14:49.948,"[5.5999999046, 2.9000000954, 3.5999999046, 1.2...",1.0,0
7,2024-03-27 23:14:49.948,"[6.9000000954, 3.0999999046, 5.0999999046, 2.2...",2.0,0
8,2024-03-27 23:14:49.948,"[6.1999998093, 2.2000000477, 4.5, 1.5]",1.0,0
9,2024-03-27 23:14:49.948,"[5.8000001907, 2.7000000477, 3.9000000954, 1.2...",1.0,0


### Undeploy the Pipeline

With the tutorial complete, we undeploy the pipeline and return the resources back to the cluster.

In [20]:
pipeline.undeploy()

name,xgboost-booster-multiclass-classification-softmax
created,2024-03-27 22:34:17.931111+00:00
last_updated,2024-03-27 23:14:10.652773+00:00
deployed,False
arch,None
accel,None
tags,
versions,"387a2bf3-708a-47fd-9202-6a5c62c53976, 4bca81af-6155-4216-89ef-e44faddc8dca, 41efcc13-ab48-4f3b-848e-66246d971ca1, 2f044e99-99d9-4e3f-a688-f33fbd29d01e, e7f230df-549c-4d00-9d0e-7e9c048575cc, 8de1145e-18b0-429f-b320-d7a0b85711ca, 1bda8577-7d4c-42dc-b0bc-ccaa6420bb73, a0acf209-e70a-409b-a498-be299a5cc15e, a4b6eb37-62c0-4b71-be41-3c92bd5de26b, 2be29333-251e-4d84-80c0-49bc5509137f, 1ccc9a25-9700-4207-aa45-9fab2552bba5, e57a2fbe-df29-432b-9927-2337a794cc20, 65d0fdbc-64d6-4ca7-9c40-95503eeef4f6, a29fb90d-5fed-42c9-9df2-bd32c19814da"
steps,xgboost-booster-multiclass-classification-softmax
published,False
